In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys

1. Leer Datos

Datos : Titanic

In [42]:
train_titanic = pd.read_csv('titanic_train.csv', sep=',')
test_t = pd.read_csv('titanic_test.csv', sep=',')
gender_sub = pd.read_csv('gender_submission.csv', sep=',')
gender_sub.PassengerId = gender_sub.PassengerId.astype(float)

train_titanic = train_titanic.replace('Q', 0)
train_titanic = train_titanic.replace('S', 1)
train_titanic = train_titanic.replace('C', 2)
train_titanic = train_titanic.replace('male', 0)
train_titanic = train_titanic.replace('female', 1)
train_titanic = np.array(train_titanic.loc[:,['Sex','Age','Fare','Embarked', 'Survived']])
train_titanic = train_titanic[~np.isnan(train_titanic).any(axis=1)]

print("Titanic Train Shape : ", train_titanic.shape)

test1_t = pd.merge(test_t, gender_sub,  how='left', on='PassengerId')
test1_t = test1_t.replace('Q', 0)
test1_t = test1_t.replace('S', 1)
test1_t = test1_t.replace('C', 2)
test1_t = test1_t.replace('male', 0)
test1_t = test1_t.replace('female', 1)

print("Titanic Test Shape : ", train_titanic.shape)

test_titanic = np.array(test1_t.loc[:,['Sex','Age','Fare','Embarked', 'Survived']])
test_titanic = test_titanic[~np.isnan(test_titanic).any(axis=1)]

titanic = np.concatenate((train_titanic, test_titanic), axis=0)
titanic = pd.DataFrame(data=titanic, columns=['Sex','Age','Fare','Embarked', 'Survived'])

print("Titanic Shape: ", titanic.shape)

titanic.to_csv('titanic_all.csv',index=False)


Titanic Train Shape :  (712, 5)
Titanic Test Shape :  (712, 5)
Titanic Shape:  (1043, 5)


Datos : Glass

Atributos:
1. Número de identificación: 1 a 214
2. RI: índice de refracción
3. Na: Sodio (unidad de medida: porcentaje en peso en el óxido correspondiente, al igual que los atributos 4-10)
4. Mg: magnesio
5. Al: Aluminio
6. Si: silicio
7. K: Potasio
8. Ca: calcio
9. Ba: bario
10. Fe: Hierro
11. Tipo de vidrio: (atributo de clase)
     1. building_windows_float_processed
     2. building_windows_non_float_processed
     3. vehicle_windows_float_processed
     4. vehicle_windows_non_float_processed (ninguno en esta base de datos)
     5. contenedores
     6. vajilla
     7. faros
    
    
Distribución de clases: (de un total de 214 instancias)

1. 163 Vidrio de ventana (ventanas de edificios y ventanas de vehículos)
     1. 87 flotador procesado
         - 70 ventanas de construcción
         - 17 ventanas de vehículos
     2. 76 procesado sin flotación
         - 76 ventanas de construcción
         - 0 ventanas de vehículos
2. 51 Vidrio no de ventana
     - 13 contenedores
     - 9 vajillas
     - 29 faros

In [43]:
glass = pd.read_csv('glass.csv')
glass.columns = ['id','ri','na','mg','al','si','k','ca','ba','fe','class']
print("Glass shape: ", glass.shape)

Glass shape:  (213, 11)


In [71]:
iris_data = pd.read_csv('iris.csv')
iris_data = iris_data.replace('Setosa', 0)
iris_data = iris_data.replace('Versicolor', 1)
iris_data = iris_data.replace('Virginica', 2)
iris_data.to_csv('iris_all.csv',index=False)
print("Iris shape: ", iris.shape)

     sepal.length  sepal.width  petal.length  petal.width  variety
0             5.1          3.5           1.4          0.2        0
1             4.9          3.0           1.4          0.2        0
2             4.7          3.2           1.3          0.2        0
3             4.6          3.1           1.5          0.2        0
4             5.0          3.6           1.4          0.2        0
..            ...          ...           ...          ...      ...
145           6.7          3.0           5.2          2.3        2
146           6.3          2.5           5.0          1.9        2
147           6.5          3.0           5.2          2.0        2
148           6.2          3.4           5.4          2.3        2
149           5.9          3.0           5.1          1.8        2

[150 rows x 5 columns]
Iris shape:  (150, 5)


In [44]:
def Leer_Datos(file_name):
    return pd.read_csv(file_name)

2. Normalizar Datos

In [45]:
def Normalizar_Datos(data):
    mean_data = np.mean(data)
    standard_dev = np.std(data)
    data = data - mean_data
    data = data / standard_dev
    return data, mean_data, standard_dev


In [46]:
def Sigmoidal(z):
    return 1.0 / (1.0+np.exp(-z))

In [47]:
def dSigmoidal(Z):
    s = Sigmoidal(Z)
    return np.multiply(s, 1 - s)

In [48]:
def Calcular_Funcion_Costo(X, y):
    result = np.sum((y * np.log(X)) + ((1 - y) * np.log(1 - X)))
    result = (-result) / y.shape[1]
    return result

In [49]:
def Activacion(X, W, b):
    Z = np.matmul(W, X) + b
    A = Sigmoidal(Z)
    return A, Z

In [50]:
def Calcular_Accuracy(X, y, W_dict, b_dict):
    _, A_dict = Forward(X, W_dict, b_dict)
    A_L = A_dict["A%d" % (len(A_dict) - 1)]
    result = A_L > 0.5
    result = np.logical_xor(np.logical_not(result), y)
    return np.sum(result) / y.shape[1]

In [61]:
def sep(data):
    n = data.shape[0]
    X = data[:n - 1, :]
    y = data[n - 1:, :]
    return X, y


def Crear_k_folds(data, k):
    size_fold = int(data.shape[1] / k)
    remainder_size_fold = int(data.shape[1] % k)
    data = data[:, :data.shape[1] - remainder_size_fold]
    k_folds = []
    idx_col = 0
    for i in range(k):
        X, y = sep(data[:, idx_col:idx_col + size_fold])
        k_folds.append({"X": X, "y": y})
        idx_col += size_fold
    return k_folds, size_fold


def Crear_Entrenamiento_Prueba(data):
    num_cols = data.shape[1]
    train_percentage = 0.7
    col_split_data = int(num_cols * train_percentage)
    training, test = data[:, :col_split_data], data[:, col_split_data:]
    return training, test


def Crear_Bias(n_actual):
    return np.random.rand(n_actual, 1)


def Crear_Pesos(n_anterior, n_actual):
    return np.random.rand(n_actual, n_anterior)


def Crear_W_b_dict(model_mlp):
    W_dict = {}
    b_dict = {}
    for l in range(len(model_mlp) - 1):
        W_dict["W%d" % (l + 1)] = Crear_Pesos(model_mlp[l], model_mlp[l + 1])
        b_dict["b%d" % (l + 1)] = Crear_Bias(model_mlp[l + 1])
    return W_dict, b_dict




def Forward(X, W_dict, b_dict):
    A = X
    Z_dict = {}
    A_dict = {"A0": A}
    for l in range(len(W_dict)):
        A, Z = Activacion(A, W_dict["W%d" % (l + 1)], b_dict["b%d" % (l + 1)])
        Z_dict["Z%d" % (l + 1)] = Z
        A_dict["A%d" % (l + 1)] = A
    return Z_dict, A_dict


def get_dC_dA_L(A_L, y):
    return -np.divide(y, A_L) + np.divide(1 - y, 1 - A_L)


def Backward(W_dict, b_dict, Z_dict, A_dict, y, learn_rate):
    m = A_dict["A0"].shape[1]
    dA = get_dC_dA_L(A_dict["A%d" % (len(A_dict) - 1)], y)
    for l in reversed(range(len(W_dict))):
        dZ = np.multiply(dA, dSigmoidal(Z_dict["Z%d" % (l + 1)]))
        dW = np.divide(np.dot(dZ, A_dict["A%d" % (l)].T), m)
        db = np.divide(np.sum(dZ, axis=1, keepdims=True), m)
        dA = np.dot(W_dict["W%d" % (l + 1)].T, dZ)
        W_dict["W%d" % (l + 1)] = W_dict["W%d" % (l + 1)] - np.multiply(learn_rate, dW)
        b_dict["b%d" % (l + 1)] = b_dict["b%d" % (l + 1)] - np.multiply(learn_rate, db)
    return W_dict, b_dict


def Gradiente_Descendiente(X, y, W_dict, b_dict, num_iter, learn_rate):
    costs = np.zeros(num_iter)
    for i in range(num_iter):
        Z_dict, A_dict = Forward(X, W_dict, b_dict)
        W_dict, b_dict = Backward(W_dict, b_dict, Z_dict, A_dict, y, learn_rate)
        costs[i] = Calcular_Funcion_Costo(A_dict["A%d" % (len(A_dict) - 1)], y)
    return W_dict, b_dict, costs





In [58]:

data_files = ["titanic_all.csv"]
learn_rates = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4]
num_iters = [500, 1000, 1500, 2000, 2500, 3000, 3500]
list_hidden_layers = [[], [2], [3, 2], [4, 3, 2]]

num_iters_label = num_iters.copy()
num_iters_label.insert(0, "Alpha \ N° Ite")

k = 3

for name in data_files:
    for hidden_layers in list_hidden_layers:
        result_table = [learn_rates]
        data = Leer_Datos(name)
        data_np = data.values
        np.random.shuffle(data_np)
        transposed_data = np.transpose(data_np)
        X, y = sep(transposed_data)
        norm_data_X, mean_data_X, standard_dev_X = Normalizar_Datos(X)
        norm_data = np.concatenate((norm_data_X, y), axis=0)

        print(norm_data.shape)
        k_folds, size_fold = Crear_k_folds(norm_data, k)
        for num_iter in num_iters:
            print("ITERACIONES: ", num_iter)
            learn_rate_row = []
            for learn_rate in learn_rates:
                print("LR: ", learn_rate)
                acc_test_total = 0.0
                for i in range(k):
                    X_train = np.zeros((norm_data.shape[0] - 1, size_fold * (k - 1)))
                    X_test = np.zeros((norm_data.shape[0] - 1, size_fold))
                    y_train = np.zeros((1, size_fold * (k - 1)))
                    y_test = np.zeros((1, size_fold))
                    count_sz_fold = 0
                    for j in range(k):
                        if j == i:
                            X_test = k_folds[i]['X']
                            y_test = k_folds[i]['y']
                        else:
                            X_train[:, count_sz_fold:count_sz_fold + size_fold] = k_folds[j]['X']
                            y_train[:, count_sz_fold:count_sz_fold + size_fold] = k_folds[j]['y']
                            count_sz_fold += size_fold

                    model_mlp = hidden_layers.copy()
                    model_mlp.insert(0, X_train.shape[0])
                    model_mlp.append(1)
                    
                   

                    W_dict, b_dict = Crear_W_b_dict(model_mlp)
                    W_dict, b_dict, costs = Gradiente_Descendiente(X_train, y_train, W_dict, b_dict, num_iter,
                                                                   learn_rate)
                    acc_test = Calcular_Accuracy(X_test, y_test, W_dict, b_dict)
                    acc_test_total += acc_test

                acc_test_total /= k
                print(model_mlp)
                print(acc_test_total)
                learn_rate_row.append("%.4f" % acc_test_total)
                
            result_table.append(learn_rate_row)
            

(5, 1043)
ITERACIONES:  500
LR:  0.01
[4, 1]
0.654178674351585
LR:  0.05
[4, 1]
0.6416906820365034
LR:  0.1
[4, 1]
0.6426512968299712
LR:  0.2
[4, 1]
0.6426512968299712
LR:  0.3
[4, 1]
0.6445725264169068
LR:  0.4
[4, 1]
0.6455331412103746
ITERACIONES:  1000
LR:  0.01
[4, 1]
0.6474543707973103
LR:  0.05
[4, 1]
0.6407300672430355
LR:  0.1
[4, 1]
0.6455331412103746
LR:  0.2
[4, 1]
0.6464937560038425
LR:  0.3
[4, 1]
0.6455331412103746
LR:  0.4
[4, 1]
0.6484149855907781
ITERACIONES:  1500
LR:  0.01
[4, 1]
0.643611911623439
LR:  0.05
[4, 1]
0.6407300672430355
LR:  0.1
[4, 1]
0.6464937560038425
LR:  0.2
[4, 1]
0.6455331412103746
LR:  0.3
[4, 1]
0.6474543707973103
LR:  0.4
[4, 1]
0.6493756003842459
ITERACIONES:  2000
LR:  0.01
[4, 1]
0.6426512968299712
LR:  0.05
[4, 1]
0.6416906820365034
LR:  0.1
[4, 1]
0.6455331412103746
LR:  0.2
[4, 1]
0.6484149855907781
LR:  0.3
[4, 1]
0.648414985590778
LR:  0.4
[4, 1]
0.6512968299711815
ITERACIONES:  2500
LR:  0.01
[4, 1]
0.6426512968299712
LR:  0.05
[4, 1

In [65]:
data_files = ["titanic_all.csv"]
learn_rates = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4]
num_iters = [500, 1000, 1500, 2000, 2500, 3000, 3500]
list_hidden_layers = [[], [2], [3, 2], [4, 3, 2]]

num_iters_label = num_iters.copy()
num_iters_label.insert(0, "Alpha \ N° Ite")

k = 3

for name in data_files:
    for hidden_layers in list_hidden_layers:
        result_table = [learn_rates]
        data = Leer_Datos(name)
        data_np = data.values
        np.random.shuffle(data_np)
        transposed_data = np.transpose(data_np)
        X, y = sep(transposed_data)
        norm_data_X, mean_data_X, standard_dev_X = Normalizar_Datos(X)
        norm_data = np.concatenate((norm_data_X, y), axis=0)

        print(norm_data.shape)
        k_folds, size_fold = Crear_k_folds(norm_data, k)
        for num_iter in num_iters:
            print("ITERACIONES: ", num_iter)
            learn_rate_row = []
            for learn_rate in learn_rates:
                print("LR: ", learn_rate)
                acc_test_total = 0.0
                for i in range(k):
                    X_train = np.zeros((norm_data.shape[0] - 1, size_fold * (k - 1)))
                    X_test = np.zeros((norm_data.shape[0] - 1, size_fold))
                    y_train = np.zeros((1, size_fold * (k - 1)))
                    y_test = np.zeros((1, size_fold))
                    count_sz_fold = 0
                    for j in range(k):
                        if j == i:
                            X_test = k_folds[i]['X']
                            y_test = k_folds[i]['y']
                        else:
                            X_train[:, count_sz_fold:count_sz_fold + size_fold] = k_folds[j]['X']
                            y_train[:, count_sz_fold:count_sz_fold + size_fold] = k_folds[j]['y']
                            count_sz_fold += size_fold

                    model_mlp = hidden_layers.copy()
                    model_mlp.insert(0, X_train.shape[0])
                    model_mlp.append(1)
                    
                   

                    W_dict, b_dict = Crear_W_b_dict(model_mlp)
                    W_dict, b_dict, costs = Gradiente_Descendiente(X_train, y_train, W_dict, b_dict, num_iter,
                                                                   learn_rate)
                    acc_test = Calcular_Accuracy(X_test, y_test, W_dict, b_dict)
                    acc_test_total += acc_test

                acc_test_total /= k
               
                learn_rate_row.append("%.4f" % acc_test_total)
                
            result_table.append(learn_rate_row)
            print(learn_rate_row)
        

(5, 1043)
ITERACIONES:  500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6561', '0.6513', '0.6378', '0.6446', '0.6484', '0.6484']
ITERACIONES:  1000
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6503', '0.6407', '0.6446', '0.6484', '0.6503', '0.6494']
ITERACIONES:  1500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6484', '0.6407', '0.6455', '0.6503', '0.6494', '0.6475']
ITERACIONES:  2000
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6503', '0.6436', '0.6484', '0.6513', '0.6503', '0.6494']
ITERACIONES:  2500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6465', '0.6475', '0.6494', '0.6494', '0.6494', '0.6503']
ITERACIONES:  3000
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6436', '0.6475', '0.6503', '0.6484', '0.6484', '0.6523']
ITERACIONES:  3500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6417', '0.6494', '0.6503', '0.6494', '0.6503', '0.6542']
(5, 1043)
ITERACIONES:  500
LR:  0.01
L

In [72]:
data_files = ["iris_all.csv"]
learn_rates = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4]
num_iters = [500, 1000, 1500, 2000, 2500, 3000, 3500]
list_hidden_layers = [[], [2], [3, 2], [4, 3, 2]]

num_iters_label = num_iters.copy()
num_iters_label.insert(0, "Alpha \ N° Ite")

k = 3

for name in data_files:
    for hidden_layers in list_hidden_layers:
        result_table2 = [learn_rates]
        data = Leer_Datos(name)
        data_np = data.values
        np.random.shuffle(data_np)
        transposed_data = np.transpose(data_np)
        X, y = sep(transposed_data)
        norm_data_X, mean_data_X, standard_dev_X = Normalizar_Datos(X)
        norm_data = np.concatenate((norm_data_X, y), axis=0)

        print(norm_data.shape)
        k_folds, size_fold = Crear_k_folds(norm_data, k)
        for num_iter in num_iters:
            print("ITERACIONES: ", num_iter)
            learn_rate_row = []
            for learn_rate in learn_rates:
                print("LR: ", learn_rate)
                acc_test_total = 0.0
                for i in range(k):
                    X_train = np.zeros((norm_data.shape[0] - 1, size_fold * (k - 1)))
                    X_test = np.zeros((norm_data.shape[0] - 1, size_fold))
                    y_train = np.zeros((1, size_fold * (k - 1)))
                    y_test = np.zeros((1, size_fold))
                    count_sz_fold = 0
                    for j in range(k):
                        if j == i:
                            X_test = k_folds[i]['X']
                            y_test = k_folds[i]['y']
                        else:
                            X_train[:, count_sz_fold:count_sz_fold + size_fold] = k_folds[j]['X']
                            y_train[:, count_sz_fold:count_sz_fold + size_fold] = k_folds[j]['y']
                            count_sz_fold += size_fold

                    model_mlp = hidden_layers.copy()
                    model_mlp.insert(0, X_train.shape[0])
                    model_mlp.append(1)
                    
                   

                    W_dict, b_dict = Crear_W_b_dict(model_mlp)
                    W_dict, b_dict, costs = Gradiente_Descendiente(X_train, y_train, W_dict, b_dict, num_iter,
                                                                   learn_rate)
                    acc_test = Calcular_Accuracy(X_test, y_test, W_dict, b_dict)
                    acc_test_total += acc_test

                acc_test_total /= k
               
                learn_rate_row.append("%.4f" % acc_test_total)
                
            result_table2.append(learn_rate_row)
            print(learn_rate_row)

(5, 150)
ITERACIONES:  500
LR:  0.01
LR:  0.05
LR:  0.1


<ipython-input-61-17f7f0dedc20>:60: RuntimeWarning: divide by zero encountered in true_divide
  return -np.divide(y, A_L) + np.divide(1 - y, 1 - A_L)
<ipython-input-61-17f7f0dedc20>:67: RuntimeWarning: invalid value encountered in multiply
  dZ = np.multiply(dA, dSigmoidal(Z_dict["Z%d" % (l + 1)]))
<ipython-input-48-42c5fb3bf39c>:2: RuntimeWarning: divide by zero encountered in log
  result = np.sum((y * np.log(X)) + ((1 - y) * np.log(1 - X)))
<ipython-input-50-1ff56e0783f8>:4: RuntimeWarning: invalid value encountered in greater
  result = A_L > 0.5


LR:  0.2
LR:  0.3
LR:  0.4
['0.8533', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  1000
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.8400', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  1500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.8533', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  2000
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.5467', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  2500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.3333', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  3000
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.3333', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  3500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.3333', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
(5, 150)
ITERACIONES:  500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4


<ipython-input-61-17f7f0dedc20>:60: RuntimeWarning: invalid value encountered in true_divide
  return -np.divide(y, A_L) + np.divide(1 - y, 1 - A_L)
<ipython-input-48-42c5fb3bf39c>:2: RuntimeWarning: invalid value encountered in multiply
  result = np.sum((y * np.log(X)) + ((1 - y) * np.log(1 - X)))


['0.6667', '0.6667', '0.5333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  1000
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6667', '0.6667', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  1500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6667', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  2000
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6667', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  2500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6667', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  3000
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6667', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
ITERACIONES:  3500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6667', '0.3333', '0.3333', '0.3333', '0.3333', '0.3333']
(5, 150)
ITERACIONES:  500
LR:  0.01
LR:  0.05
LR:  0.1
LR:  0.2
LR:  0.3
LR:  0.4
['0.6667', '0.6667', '0.6667', '0.3333',